In [1]:
# need to split everything by sales rep, probably straight after raw_df import

In [8]:
pip install nbformat

Note: you may need to restart the kernel to use updated packages.


In [9]:
%run "/Users/mennahjafar/Desktop/scalr/backend/scalr_v2_functions.ipynb"

Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import *
from pyspark.sql.functions import col, when, min as min_
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.types import IntegerType, FloatType
from pyspark.sql.window import Window
from functools import reduce
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
import delta
import numpy as np
import statsmodels.api as sm
from itertools import combinations
from scipy import integrate
import matplotlib.pyplot as plt
from scipy.stats import poisson
import math
from flask import Flask, jsonify

In [11]:
def create_spark_session():
    from pyspark.sql import SparkSession
    spark = SparkSession.builder \
        .appName("YourAppName") \
        .getOrCreate()
    return spark

# Initialize SparkSession and make it globally available
spark = create_spark_session()

from pyspark.sql.types import StructField, StructType, FloatType

def load_and_prepare_data(file_location, file_type="csv", infer_schema="true", first_row_is_header="true", delimiter=",", temp_table_name="raw_data"):
    df = spark.read.format(file_type) \
        .option("inferSchema", infer_schema) \
        .option("header", first_row_is_header) \
        .option("sep", delimiter) \
        .load(file_location)  # This line completes the DataFrame reading process

    df.createOrReplaceTempView(temp_table_name)
    return df


# Correct usage:
file_location = "/Users/mennahjafar/Desktop/scalr/backend/Final_Corrected_Complex_datasetv2_v2.csv"
df = load_and_prepare_data(file_location)

#modifying the opportunity stage column to replace Closed with Closed Won or Closed Lost depending on how it's ended
df = df.withColumn("Opportunity Stage", 
                            when((col("Opportunity Stage") == "Closed") & (col("Closed Won") == 1), "Closed Won")
                            .when((col("Opportunity Stage") == "Closed") & (col("Closed Lost") == 1), "Closed Lost")
                            .otherwise(col("Opportunity Stage")))


df.show()




+----------+-------+------+-----------------+----------+-----------+-------------+-------------+--------------+------------------+------------------+----------------------+------------------+------------------+---------------------------+----------------------+-----------------------+------------------+-----------------+----------------+---------------+
|    PERIOD|OwnerId|DealID|Opportunity Stage|Closed won|Closed lost|Inbound Calls|Inbound Email|Outbound Email|Meetings Scheduled|Lead Response Time|Average Contract Value|Sales Cycle Length|Linkedin Messaging|Daily opportunity movements|Outbound leads created|Contacts per/in company|Sales by territory|Sales by industry|Sales by channel|Number of Demos|
+----------+-------+------+-----------------+----------+-----------+-------------+-------------+--------------+------------------+------------------+----------------------+------------------+------------------+---------------------------+----------------------+-----------------------+---

In [25]:
import pandas as pd
import json

# Convert DataFrame to JSON object
json_array = df.toJSON().collect()
print("JSON array:",json_array)

json_object = json.loads(json_array[0])
print("JSON array:",json_object)



JSON array: ['{"PERIOD":"2021-11-01","OwnerId":1,"DealID":1,"Opportunity Stage":"Pitching","Closed won":0,"Closed lost":0,"Inbound Calls":1,"Inbound Email":3,"Outbound Email":1,"Meetings Scheduled":1,"Lead Response Time":8,"Average Contract Value":4307.5,"Sales Cycle Length":0,"Linkedin Messaging":7,"Daily opportunity movements":2,"Outbound leads created":0,"Contacts per/in company":1,"Sales by territory":1,"Sales by industry":2,"Sales by channel":1,"Number of Demos":3}', '{"PERIOD":"2021-01-17","OwnerId":1,"DealID":1,"Opportunity Stage":"Negotiation","Closed won":0,"Closed lost":0,"Inbound Calls":6,"Inbound Email":2,"Outbound Email":3,"Meetings Scheduled":0,"Lead Response Time":36,"Average Contract Value":4307.5,"Sales Cycle Length":6,"Linkedin Messaging":1,"Daily opportunity movements":2,"Outbound leads created":3,"Contacts per/in company":6,"Sales by territory":1,"Sales by industry":2,"Sales by channel":1,"Number of Demos":1}', '{"PERIOD":"2021-02-02","OwnerId":1,"DealID":1,"Opportu

In [26]:
from flask import Flask, jsonify
from threading import Thread
import pandas as pd

app = Flask(__name__)


@app.route('/data')
def get_data():
    # Convert DataFrame to JSON
    json_array = df.toJSON().collect()
    print("data requested: ", json_array)
    return json_array


def run_flask_app():
    app.run(port=0)  # Use port=0 for dynamic port assignment

# Start the Flask app in a separate thread
t = Thread(target=run_flask_app)
t.start()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:56213
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:56213
Press CTRL+C to quit
INFO:werkzeug:Press CTRL+C to quit
127.0.0.1 - - [08/Apr/2024 21:12:51] "GET /data HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Apr/2024 21:12:51] "GET /data HTTP/1.1" 200 -


data requested:  ['{"PERIOD":"2021-11-01","OwnerId":1,"DealID":1,"Opportunity Stage":"Pitching","Closed won":0,"Closed lost":0,"Inbound Calls":1,"Inbound Email":3,"Outbound Email":1,"Meetings Scheduled":1,"Lead Response Time":8,"Average Contract Value":4307.5,"Sales Cycle Length":0,"Linkedin Messaging":7,"Daily opportunity movements":2,"Outbound leads created":0,"Contacts per/in company":1,"Sales by territory":1,"Sales by industry":2,"Sales by channel":1,"Number of Demos":3}', '{"PERIOD":"2021-01-17","OwnerId":1,"DealID":1,"Opportunity Stage":"Negotiation","Closed won":0,"Closed lost":0,"Inbound Calls":6,"Inbound Email":2,"Outbound Email":3,"Meetings Scheduled":0,"Lead Response Time":36,"Average Contract Value":4307.5,"Sales Cycle Length":6,"Linkedin Messaging":1,"Daily opportunity movements":2,"Outbound leads created":3,"Contacts per/in company":6,"Sales by territory":1,"Sales by industry":2,"Sales by channel":1,"Number of Demos":1}', '{"PERIOD":"2021-02-02","OwnerId":1,"DealID":1,"Op

127.0.0.1 - - [08/Apr/2024 21:12:51] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [08/Apr/2024 21:12:51] "GET /favicon.ico HTTP/1.1" 404 -
24/04/09 01:04:33 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 908445 ms exceeds timeout 120000 ms
24/04/09 01:04:33 WARN SparkContext: Killing executors is not supported by current scheduler.
24/04/09 01:04:34 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lz

In [7]:
raw_df = spark.read.table('raw_data')

display(raw_df)

DataFrame[PERIOD: date, OwnerId: int, DealID: int, Opportunity stage: string, Closed won: int, Closed lost: int, Inbound Calls: int, Inbound Email: int, Outbound Email: int, Meetings Scheduled: int, Lead Response Time: int, Average Contract Value: double, Sales Cycle Length: int, Linkedin Messaging: int, Daily opportunity movements: int, Outbound leads created: int, Contacts per/in company: int, Sales by territory: int, Sales by industry: int, Sales by channel: int, Number of Demos: int]

In [10]:
def get_data_by_owner_and_deal_pyspark(owner_id, deal_id=None):
    """
    Filters the dataset for a specific OwnerId in PySpark, optionally filters by DealID,
    and returns datasets for each DealID (if DealID is not specified) sorted chronologically.
    
    Parameters:
    - owner_id: The OwnerId to filter by.
    - deal_id: Optional. The specific DealID to filter by. If None, returns data for all DealIDs for the owner.
    
    Returns:
    - A dictionary of DataFrames, with DealID as keys, each DataFrame sorted chronologically.
      If a specific DealID is provided, returns a single DataFrame for that DealID.
    """
    # Filter by OwnerId
    owner_data = df.filter(df['OwnerId'] == owner_id)
    
    if deal_id is not None:
        # If a specific DealID is provided, further filter by DealID
        deal_data = owner_data.filter(owner_data['DealID'] == deal_id).orderBy('PERIOD')
        return {deal_id: deal_data}
    else:
        # If no specific DealID is provided, separate by DealID and sort each subset
        deal_ids = owner_data.select('DealID').distinct().collect()
        sorted_deal_datasets = {}
        
        for row in deal_ids:
            deal_id = row['DealID']
            deal_data = owner_data.filter(owner_data['DealID'] == deal_id).orderBy('PERIOD')
            sorted_deal_datasets[deal_id] = deal_data
        
        return sorted_deal_datasets




def print_data_for_owner(deal_datasets, deal_id=None):
    """
    Prints the datasets for a specific OwnerId. If a DealID is specified, it prints only that dataset;
    otherwise, it prints all datasets for the OwnerId.
    
    Parameters:
    - deal_datasets: A dictionary of DataFrames with DealID as keys, each DataFrame sorted chronologically.
    - deal_id: Optional. The specific DealID to print data for. If None, prints data for all DealIDs.
    """
    if deal_id is not None:
        # Print data for the specified DealID
        if deal_id in deal_datasets:
            print(f"Data for DealID={deal_id}:\n")
            deal_datasets[deal_id].show()
        else:
            print(f"No data found for DealID={deal_id}.")
    else:
        # Print data for all DealIDs
        for did, data in deal_datasets.items():
            print(f"Data for DealID={did}:\n")
            data.show()

# Example usage:
the_owner_id= 2
# Retrieve data for OwnerId = 2 for all DealIDs
data_for_owner = get_data_by_owner_and_deal_pyspark(the_owner_id)

# Print data for all DealIDs for OwnerId = 2
print_data_for_owner(data_for_owner)

# Print data for a specific DealID (e.g., DealID = 3) for OwnerId = 2
print_data_for_owner(data_for_owner, deal_id=3)


Data for DealID=255:

+----------+-------+------+-----------------+----------+-----------+-------------+-------------+--------------+------------------+------------------+----------------------+------------------+------------------+---------------------------+----------------------+-----------------------+------------------+-----------------+----------------+---------------+
|    PERIOD|OwnerId|DealID|Opportunity Stage|Closed won|Closed lost|Inbound Calls|Inbound Email|Outbound Email|Meetings Scheduled|Lead Response Time|Average Contract Value|Sales Cycle Length|Linkedin Messaging|Daily opportunity movements|Outbound leads created|Contacts per/in company|Sales by territory|Sales by industry|Sales by channel|Number of Demos|
+----------+-------+------+-----------------+----------+-----------+-------------+-------------+--------------+------------------+------------------+----------------------+------------------+------------------+---------------------------+----------------------+-----

In [11]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, lead

# Specify the OwnerID you're interested in
selected_owner_id = 1  # Replace with the actual OwnerID you're analyzing

# Filter the dataset for the selected OwnerID
df_owner_specific = df.filter(col("OwnerId") == selected_owner_id)

# Define the window specification, partitioned by DealID and ordered by PERIOD
window_spec = Window.partitionBy("DealID").orderBy("PERIOD")

# Create a new column for the next Opportunity Stage
df_owner_specific = df_owner_specific.withColumn("NextOpportunityStage", lead("Opportunity Stage").over(window_spec))

# Filter rows to include only those where the current stage is "Interaction" and a next stage exists
df_interaction_transitions = df_owner_specific.filter((col("Opportunity Stage") == "Interaction") & col("NextOpportunityStage").isNotNull())

# Filter for records where the current stage is "Pitching"
df_pitching_transitions = df_owner_specific.filter((col("Opportunity Stage") == "Pitching") & col("NextOpportunityStage").isNotNull())

df_negotiation_transitions = df_owner_specific.filter((col("Opportunity Stage") == "Negotiation") & col("NextOpportunityStage").isNotNull())

df_commitment_transitions = df_owner_specific.filter((col("Opportunity Stage") == "Commitment") & col("NextOpportunityStage").isNotNull())

df_closed_won_transitions = df_owner_specific.filter((col("Opportunity Stage") == "Closed Won") & col("NextOpportunityStage").isNotNull())

df_closed_lost_transitions = df_owner_specific.filter((col("Opportunity Stage") == "Closed Lost") & col("NextOpportunityStage").isNotNull())


# Show the result to verify
df_interaction_transitions.show(5)
df_pitching_transitions.show(5)
df_negotiation_transitions.show(5)
df_commitment_transitions.show(5)
df_closed_won_transitions.show(5)
df_closed_lost_transitions.show(5)


+----------+-------+------+-----------------+----------+-----------+-------------+-------------+--------------+------------------+------------------+----------------------+------------------+------------------+---------------------------+----------------------+-----------------------+------------------+-----------------+----------------+---------------+--------------------+
|    PERIOD|OwnerId|DealID|Opportunity Stage|Closed won|Closed lost|Inbound Calls|Inbound Email|Outbound Email|Meetings Scheduled|Lead Response Time|Average Contract Value|Sales Cycle Length|Linkedin Messaging|Daily opportunity movements|Outbound leads created|Contacts per/in company|Sales by territory|Sales by industry|Sales by channel|Number of Demos|NextOpportunityStage|
+----------+-------+------+-----------------+----------+-----------+-------------+-------------+--------------+------------------+------------------+----------------------+------------------+------------------+---------------------------+--------

In [12]:
from pyspark.ml.feature import VectorAssembler, StringIndexer

# List of predictor columns based on the metrics you've mentioned
predictor_columns = [
    "Inbound Calls", "Inbound Email", "Outbound Email", "Meetings Scheduled",
    "Lead Response Time", "Average Contract Value", "Sales Cycle Length",
    "Linkedin Messaging", "Daily opportunity movements", "Outbound leads created",
    "Contacts per/in company", "Sales by territory", "Sales by industry",
    "Sales by channel", "Number of Demos"
]

# Assemble predictors into a feature vector
assembler_interaction = VectorAssembler(inputCols=predictor_columns, outputCol="features_interaction")
df_interaction_features = assembler_interaction.transform(df_interaction_transitions)

# Assemble predictors into a feature vector for the "Pitching" stage
assembler_pitching = VectorAssembler(inputCols=predictor_columns, outputCol="features_pitching")
df_pitching_features = assembler_pitching.transform(df_pitching_transitions)

assembler_negotiation = VectorAssembler(inputCols=predictor_columns, outputCol="features_negotiation")
df_negotiation_features = assembler_negotiation.transform(df_negotiation_transitions)

assembler_commitment = VectorAssembler(inputCols=predictor_columns, outputCol="features_commitment")
df_commitment_features = assembler_commitment.transform(df_commitment_transitions)

assembler_closed_won = VectorAssembler(inputCols=predictor_columns, outputCol="features_closed_won")
df_closed_won_features = assembler_closed_won.transform(df_closed_won_transitions)

assembler_closed_lost = VectorAssembler(inputCols=predictor_columns, outputCol="features_closed_lost")
df_closed_lost_features = assembler_closed_lost.transform(df_closed_lost_transitions)





indexer_interaction = StringIndexer(inputCol="NextOpportunityStage", outputCol="label_interaction")
indexer_model_interaction = indexer_interaction.fit(df_interaction_features)
df_interaction_prepared = indexer_model_interaction.transform(df_interaction_features)
# Encode the "NextOpportunityStage" for the "Pitching" stage
indexer_pitching = StringIndexer(inputCol="NextOpportunityStage", outputCol="label_pitching")
indexer_model_pitching = indexer_pitching.fit(df_pitching_features)
df_pitching_prepared = indexer_model_pitching.transform(df_pitching_features)

indexer_negotiation = StringIndexer(inputCol="NextOpportunityStage", outputCol="label_negotiation")
indexer_model_negotiation = indexer_negotiation.fit(df_negotiation_features)
df_negotiation_prepared = indexer_model_negotiation.transform(df_negotiation_features)

indexer_commitment = StringIndexer(inputCol="NextOpportunityStage", outputCol="label_commitment")
indexer_model_commitment = indexer_commitment.fit(df_commitment_features)
df_commitment_prepared = indexer_model_commitment.transform(df_commitment_features)

indexer_closed_won = StringIndexer(inputCol="NextOpportunityStage", outputCol="label_closed_won")
indexer_model_closed_won = indexer_closed_won.fit(df_closed_won_features)
df_closed_won_prepared = indexer_model_closed_won.transform(df_closed_won_features)

indexer_closed_lost = StringIndexer(inputCol="NextOpportunityStage", outputCol="label_closed_lost")
indexer_model_closed_lost = indexer_closed_lost.fit(df_closed_lost_features)
df_closed_lost_prepared = indexer_model_closed_lost.transform(df_closed_lost_features)








# Correct approach after fitting StringIndexer for the "Interaction" stage


# You've already done it correctly for the "Pitching" stage




# Optional: Extract label to stage mapping, which could be useful for interpreting the model's output later
label_to_stage_mapping_interaction = dict(enumerate(indexer_model_interaction.labels))
# Generate label to stage mapping for the "Pitching" stage
label_to_stage_mapping_pitching = dict(enumerate(indexer_model_pitching.labels))

label_to_stage_mapping_negotiation = dict(enumerate(indexer_model_negotiation.labels))

label_to_stage_mapping_commitment = dict(enumerate(indexer_model_commitment.labels))

label_to_stage_mapping_closed_won = dict(enumerate(indexer_model_commitment.labels))

label_to_stage_mapping_closed_lost = dict(enumerate(indexer_model_commitment.labels))




# Let's take a look at the prepared DataFrame
df_interaction_prepared.select("features_interaction", "label_interaction").show(5)

df_pitching_prepared.select("features_pitching", "label_pitching").show(5)

df_negotiation_prepared.select("features_negotiation", "label_negotiation").show(5)

df_commitment_prepared.select("features_commitment", "label_commitment").show(5)

df_closed_won_prepared.select("features_closed_won", "label_closed_won").show(5)

df_closed_lost_prepared.select("features_closed_lost", "label_closed_lost").show(5)











24/04/07 17:18:12 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+-----------------+
|features_interaction|label_interaction|
+--------------------+-----------------+
|[6.0,0.0,2.0,0.0,...|              0.0|
|[7.0,1.0,0.0,0.0,...|              0.0|
|[8.0,1.0,4.0,4.0,...|              0.0|
|[2.0,4.0,3.0,3.0,...|              0.0|
|[3.0,1.0,1.0,3.0,...|              0.0|
+--------------------+-----------------+
only showing top 5 rows

+--------------------+--------------+
|   features_pitching|label_pitching|
+--------------------+--------------+
|[6.0,0.0,2.0,4.0,...|           0.0|
|[3.0,1.0,3.0,3.0,...|           0.0|
|[1.0,1.0,0.0,3.0,...|           0.0|
|[2.0,0.0,0.0,0.0,...|           0.0|
|[1.0,1.0,2.0,2.0,...|           2.0|
+--------------------+--------------+
only showing top 5 rows

+--------------------+-----------------+
|features_negotiation|label_negotiation|
+--------------------+-----------------+
|[6.0,2.0,3.0,0.0,...|              0.0|
|[2.0,0.0,1.0,2.0,...|              0.0|
|[6.0,3.0,1.0,1.0,...|            

In [13]:
from pyspark.ml.classification import LogisticRegression


lr_interaction = LogisticRegression(featuresCol="features_interaction", labelCol="label_interaction", family="multinomial")
model_interaction = lr_interaction.fit(df_interaction_prepared) 


print("Coefficients interaction: \n" + str(model_interaction.coefficientMatrix))
print("Intercepts interaction: " + str(model_interaction.interceptVector))

lr_pitching = LogisticRegression(featuresCol="features_pitching", labelCol="label_pitching", family="multinomial")
model_pitching = lr_pitching.fit(df_pitching_prepared)  


print("Coefficients pitching: \n" + str(model_pitching.coefficientMatrix))
print("Intercepts pitching: " + str(model_pitching.interceptVector))

lr_negotiation = LogisticRegression(featuresCol="features_negotiation", labelCol="label_negotiation", family="multinomial")
model_negotiation = lr_negotiation.fit(df_negotiation_prepared)  # Use df_pitching_prepared


print("Coefficients negotiation: \n" + str(model_negotiation.coefficientMatrix))
print("Intercepts negotiation: " + str(model_negotiation.interceptVector))

lr_commitment = LogisticRegression(featuresCol="features_commitment", labelCol="label_commitment", family="multinomial")
model_commitment = lr_commitment.fit(df_commitment_prepared)  # Use df_pitching_prepared

# Print the coefficients and intercepts for the multinomial logistic regression
# Coefficients and intercepts should be printed from the fitted model, not the uninitialized model
print("Coefficients commitment: \n" + str(model_commitment.coefficientMatrix))
print("Intercepts commitment: " + str(model_commitment.interceptVector))

lr_closed_won = LogisticRegression(featuresCol="features_closed_won", labelCol="label_closed_won", family="multinomial")
model_closed_won = lr_closed_won.fit(df_closed_won_prepared)  # Use df_pitching_prepared

# Print the coefficients and intercepts for the multinomial logistic regression
# Coefficients and intercepts should be printed from the fitted model, not the uninitialized model
print("Coefficients Closed Won: \n" + str(model_closed_won.coefficientMatrix))
print("Intercepts Closed Won: " + str(model_closed_won.interceptVector))

lr_closed_lost = LogisticRegression(featuresCol="features_closed_lost", labelCol="label_closed_lost", family="multinomial")
model_closed_lost = lr_closed_lost.fit(df_closed_lost_prepared)  # Use df_pitching_prepared

# Print the coefficients and intercepts for the multinomial logistic regression
# Coefficients and intercepts should be printed from the fitted model, not the uninitialized model
print("Coefficients Closed Lost: \n" + str(model_closed_lost.coefficientMatrix))
print("Intercepts Closed Lost: " + str(model_closed_lost.interceptVector))


24/04/07 17:18:24 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


Coefficients interaction: 
DenseMatrix([[ 1.79689090e+00, -3.58382672e+00,  4.08912698e+00,
              -1.10149251e+00, -2.33487831e-01, -1.58317452e-04,
               1.82813094e-01,  5.92752677e+00, -1.15466101e+00,
              -5.03368243e-01, -3.04753763e+00,  3.66945128e-01,
               1.85517740e+00, -1.34975594e+00, -8.81233920e-01],
             [-1.15630500e+00,  8.23316725e+00, -2.37336021e+00,
               6.11899413e-01, -4.69855031e-01, -3.42571998e-03,
              -1.34830905e-01, -5.77093897e+00, -9.56365227e+00,
               1.19498457e+00,  2.62824757e+00, -1.87624440e+01,
              -4.49969261e+00, -7.64066254e+00,  2.69783125e+00],
             [-7.77635774e-01,  3.13762798e+00, -5.26955406e+00,
              -5.47515526e+00, -3.37429464e-02,  3.10855683e-03,
              -3.27198549e-02,  6.87840920e-01,  8.40621411e+00,
              -2.75250013e+00, -3.72392829e+00,  1.08040572e+01,
               1.68570674e+01, -1.13723404e+01, -6.88682260e-

In [15]:
import numpy as np

# Get the coefficients in a dense format



coefficients_matrix_interaction = np.array(model_interaction.coefficientMatrix.toArray())
intercepts_interaction = model_interaction.interceptVector

# Adjusting the loop to use the label_to_stage_mapping_pitching
for i in range(coefficients_matrix_interaction.shape[0]):
    stage_name_interaction = label_to_stage_mapping_interaction.get(i, "Unknown Stage")  # Get stage name or default
    print(f"\nClass {i} (Interaction transition to Stage {stage_name_interaction}):")
    for j, feature_name in enumerate(predictor_columns):
        print(f"  {feature_name}: {coefficients_matrix_interaction[i, j]:.4f}")
    print(f"  Intercept: {intercepts_interaction[i]:.4f}")

# For the "Pitching" stage
coefficients_matrix_pitching = np.array(model_pitching.coefficientMatrix.toArray())
intercepts_pitching = model_pitching.interceptVector

# Adjusting the loop to use the label_to_stage_mapping_pitching
for i in range(coefficients_matrix_pitching.shape[0]):
    stage_name_pitching = label_to_stage_mapping_pitching.get(i, "Unknown Stage")  # Get stage name or default
    print(f"\nClass {i} (Pitching transition to Stage {stage_name_pitching}):")
    for j, feature_name in enumerate(predictor_columns):
        print(f"  {feature_name}: {coefficients_matrix_pitching[i, j]:.4f}")
    print(f"  Intercept: {intercepts_pitching[i]:.4f}")

coefficients_matrix_negotiation = np.array(model_negotiation.coefficientMatrix.toArray())
intercepts_negotiation = model_negotiation.interceptVector

# Adjusting the loop to use the label_to_stage_mapping_pitching
for i in range(coefficients_matrix_negotiation.shape[0]):
    stage_name_negotiation = label_to_stage_mapping_negotiation.get(i, "Unknown Stage")  # Get stage name or default
    print(f"\nClass {i} (Negotiation transition to Stage {stage_name_negotiation}):")
    for j, feature_name in enumerate(predictor_columns):
        print(f"  {feature_name}: {coefficients_matrix_negotiation[i, j]:.4f}")
    print(f"  Intercept: {intercepts_negotiation[i]:.4f}")

coefficients_matrix_commitment = np.array(model_commitment.coefficientMatrix.toArray())
intercepts_commitment = model_commitment.interceptVector

# Adjusting the loop to use the label_to_stage_mapping_pitching
for i in range(coefficients_matrix_commitment.shape[0]):
    stage_name_commitment = label_to_stage_mapping_commitment.get(i, "Unknown Stage")  # Get stage name or default
    print(f"\nClass {i} (Commitment transition to Stage {stage_name_commitment}):")
    for j, feature_name in enumerate(predictor_columns):
        print(f"  {feature_name}: {coefficients_matrix_commitment[i, j]:.4f}")
    print(f"  Intercept: {intercepts_commitment[i]:.4f}")

coefficients_matrix_closed_won = np.array(model_closed_won.coefficientMatrix.toArray())
intercepts_closed_won = model_closed_won.interceptVector

# Adjusting the loop to use the label_to_stage_mapping_pitching
for i in range(coefficients_matrix_closed_won.shape[0]):
    stage_name_closed_won = label_to_stage_mapping_closed_won.get(i, "Unknown Stage")  # Get stage name or default
    print(f"\nClass {i} (Closed Won transition to Stage {stage_name_closed_won}):")
    for j, feature_name in enumerate(predictor_columns):
        print(f"  {feature_name}: {coefficients_matrix_closed_won[i, j]:.4f}")
    print(f"  Intercept: {intercepts_closed_won[i]:.4f}")

coefficients_matrix_closed_lost = np.array(model_closed_lost.coefficientMatrix.toArray())
intercepts_closed_lost = model_closed_lost.interceptVector

# Adjusting the loop to use the label_to_stage_mapping_pitching
for i in range(coefficients_matrix_closed_lost.shape[0]):
    stage_name_closed_lost = label_to_stage_mapping_closed_lost.get(i, "Unknown Stage")  # Get stage name or default
    print(f"\nClass {i} (Closed Lost transition to Stage {stage_name_closed_lost}):")
    for j, feature_name in enumerate(predictor_columns):
        print(f"  {feature_name}: {coefficients_matrix_closed_lost[i, j]:.4f}")
    print(f"  Intercept: {intercepts_closed_lost[i]:.4f}")




Class 0 (Interaction transition to Stage Commitment):
  Inbound Calls: 1.7969
  Inbound Email: -3.5838
  Outbound Email: 4.0891
  Meetings Scheduled: -1.1015
  Lead Response Time: -0.2335
  Average Contract Value: -0.0002
  Sales Cycle Length: 0.1828
  Linkedin Messaging: 5.9275
  Daily opportunity movements: -1.1547
  Outbound leads created: -0.5034
  Contacts per/in company: -3.0475
  Sales by territory: 0.3669
  Sales by industry: 1.8552
  Sales by channel: -1.3498
  Number of Demos: -0.8812
  Intercept: 37.4877

Class 1 (Interaction transition to Stage Pitching):
  Inbound Calls: -1.1563
  Inbound Email: 8.2332
  Outbound Email: -2.3734
  Meetings Scheduled: 0.6119
  Lead Response Time: -0.4699
  Average Contract Value: -0.0034
  Sales Cycle Length: -0.1348
  Linkedin Messaging: -5.7709
  Daily opportunity movements: -9.5637
  Outbound leads created: 1.1950
  Contacts per/in company: 2.6282
  Sales by territory: -18.7624
  Sales by industry: -4.4997
  Sales by channel: -7.6407
  N

In [16]:
# Define dictionaries to hold coefficients and intercepts for each stage transition
coefficients_all_stages = {}
intercepts_all_stages = {}

# Define a dictionary to hold label to stage mappings for all stages
label_to_stage_mapping = {
    "Interaction": indexer_model_interaction.labels,
    "Pitching": indexer_model_pitching.labels,
    "Negotiation": indexer_model_negotiation.labels,
    "Commitment": indexer_model_commitment.labels,
    "Closed_Won": indexer_model_closed_won.labels,
    "Closed_Lost": indexer_model_closed_lost.labels
}

# Loop over each stage
for stage, labels in label_to_stage_mapping.items():
    # Extract coefficients and intercepts for the current stage transition
    coefficients_matrix_stage = np.array(vars()["model_" + stage.lower()].coefficientMatrix.toArray())
    intercepts_stage = vars()["model_" + stage.lower()].interceptVector

    # Initialize dictionaries for coefficients and intercepts of the current stage
    coefficients_stage = {}
    intercepts_stage_dict = {}

    # Initialize placeholders for missing transitions
    missing_transitions = set(label_to_stage_mapping.keys()) - {stage}

    # Adjusting the loop to use the label_to_stage_mapping for the current stage
    for i, stage_name in enumerate(labels):
        coefficients_stage[stage_name] = {}
        intercepts_stage_dict[stage_name] = intercepts_stage[i]
        print(f"\nTransition from Stage {stage} to Stage {stage_name}:")
        for j, feature_name in enumerate(predictor_columns):
            coefficients_stage[stage_name][feature_name] = coefficients_matrix_stage[i, j]
            print(f"  {feature_name}: {coefficients_matrix_stage[i, j]:.4f}")
        print(f"  Intercept: {intercepts_stage[i]:.4f}")

        # Remove current transition from missing_transitions
        missing_transitions.discard(stage_name)

    # Fill in missing transitions with zeros
    for missing_transition in missing_transitions:
        coefficients_stage[missing_transition] = {feature_name: 0.0 for feature_name in predictor_columns}
        intercepts_stage_dict[missing_transition] = 0.0

    # Store coefficients and intercepts dictionaries for the current stage transition
    coefficients_all_stages[stage] = coefficients_stage
    intercepts_all_stages[stage] = intercepts_stage_dict

# Print coefficients and intercepts dictionaries for all stages
print("\nCoefficients for all stage transitions:")
print(coefficients_all_stages)
print("\nIntercepts for all stage transitions:")
print(intercepts_all_stages)



Transition from Stage Interaction to Stage Commitment:
  Inbound Calls: 1.7969
  Inbound Email: -3.5838
  Outbound Email: 4.0891
  Meetings Scheduled: -1.1015
  Lead Response Time: -0.2335
  Average Contract Value: -0.0002
  Sales Cycle Length: 0.1828
  Linkedin Messaging: 5.9275
  Daily opportunity movements: -1.1547
  Outbound leads created: -0.5034
  Contacts per/in company: -3.0475
  Sales by territory: 0.3669
  Sales by industry: 1.8552
  Sales by channel: -1.3498
  Number of Demos: -0.8812
  Intercept: 37.4877

Transition from Stage Interaction to Stage Pitching:
  Inbound Calls: -1.1563
  Inbound Email: 8.2332
  Outbound Email: -2.3734
  Meetings Scheduled: 0.6119
  Lead Response Time: -0.4699
  Average Contract Value: -0.0034
  Sales Cycle Length: -0.1348
  Linkedin Messaging: -5.7709
  Daily opportunity movements: -9.5637
  Outbound leads created: 1.1950
  Contacts per/in company: 2.6282
  Sales by territory: -18.7624
  Sales by industry: -4.4997
  Sales by channel: -7.6407
 

In [18]:
from pyspark.sql.window import Window
import pyspark.sql.functions as F

def extract_latest_active_entries(df, owner_id=None, metrics=None):
    if metrics is None:
        metrics = [
            "Inbound Calls", "Inbound Email", "Outbound Email", "Meetings Scheduled",
            "Lead Response Time", "Average Contract Value", "Sales Cycle Length",
            "Linkedin Messaging", "Daily opportunity movements", "Outbound leads created",
            "Contacts per/in company", "Sales by territory", "Sales by industry", 
            "Sales by channel", "Number of Demos"
        ]
    
    # Optionally filter by OwnerID if specified
    if owner_id is not None:
        df = df.filter(df["OwnerID"] == owner_id)
    
    # Define the window specification
    windowSpec = Window.partitionBy("DealID").orderBy(F.desc("PERIOD"))
    
    # Use the window spec to mark the latest entry for each DealID
    df_latest = df.withColumn("Rank", F.row_number().over(windowSpec)) \
                  .filter(F.col("Rank") == 1) \
                  .drop("Rank")
    
    # Filtering out deals that have reached an absorbing state as their latest stage
    active_deals_df = df_latest.filter(~F.col("Opportunity Stage").isin(["Closed Won", "Closed Lost"]))
    
    # Selecting the deal ID, latest opportunity stage, and the metrics of interest
    active_deals_metrics = active_deals_df.select("DealID", "Opportunity Stage", *metrics)
    
    return active_deals_metrics

# Assuming df is your Spark DataFrame
active_deals_metrics_df = extract_latest_active_entries(df, selected_owner_id)

# Show the resulting DataFrame for verification
active_deals_metrics_df.show()



filtered_df = active_deals_metrics_df.filter(active_deals_metrics_df["Average Contract Value"].isNotNull())

# Select only the latest Average Contract Values
latest_avg_contract_values = filtered_df.select("Average Contract Value")

# Collect the latest Average Contract Values as a vector



# Collect the latest Average Contract Values as a list
latest_avg_contract_values_list = [row[0] for row in latest_avg_contract_values.collect()]

# Convert the list to a NumPy array
latest_avg_contract_values_array = np.array(latest_avg_contract_values_list)

print("Latest Average Contract Values (as a NumPy array):", latest_avg_contract_values_array)


+------+-----------------+-------------+-------------+--------------+------------------+------------------+----------------------+------------------+------------------+---------------------------+----------------------+-----------------------+------------------+-----------------+----------------+---------------+
|DealID|Opportunity Stage|Inbound Calls|Inbound Email|Outbound Email|Meetings Scheduled|Lead Response Time|Average Contract Value|Sales Cycle Length|Linkedin Messaging|Daily opportunity movements|Outbound leads created|Contacts per/in company|Sales by territory|Sales by industry|Sales by channel|Number of Demos|
+------+-----------------+-------------+-------------+--------------+------------------+------------------+----------------------+------------------+------------------+---------------------------+----------------------+-----------------------+------------------+-----------------+----------------+---------------+
|     1|         Pitching|            1|            3|    

In [19]:
import numpy as np

def compute_transitions(active_deals_metrics_df, coefficients_all_stages, intercepts_all_stages, deal_id=None, num_top_transitions=5):
    metrics = [
        "Inbound Calls", "Inbound Email", "Outbound Email", "Meetings Scheduled",
        "Lead Response Time", "Average Contract Value", "Sales Cycle Length",
        "Linkedin Messaging", "Daily opportunity movements", "Outbound leads created",
        "Contacts per/in company", "Sales by territory", "Sales by industry",
        "Sales by channel", "Number of Demos"
    ]
    
    results = {}

    if deal_id is not None:
        deal_ids = [deal_id]
    else:
        deal_ids = [row[0] for row in active_deals_metrics_df.select("DealID").distinct().collect()]
    
    for deal_id in deal_ids:
        variables_and_stage = active_deals_metrics_df.filter(active_deals_metrics_df.DealID == deal_id) \
                                                     .select("Opportunity Stage", *metrics) \
                                                     .collect()[0].asDict()

        current_stage = variables_and_stage["Opportunity Stage"]

        coefficients = coefficients_all_stages[current_stage]
        intercepts = intercepts_all_stages[current_stage]

        vector = np.array([variables_and_stage[metric] for metric in metrics])

        dot_product_results = []
        exponentiated_results = []
        corresponding_stages = []
        for stage_name, stage_coefficients in coefficients.items():
            dot_product_result = np.dot(list(stage_coefficients.values()), vector) + intercepts[stage_name]
            exponentiated_result = np.exp(dot_product_result)
            dot_product_results.append(dot_product_result)
            exponentiated_results.append(exponentiated_result)
            corresponding_stages.append(stage_name)

        sorted_indices = np.argsort(exponentiated_results)[::-1][:num_top_transitions]
        sum_dot_products = np.sum([np.exp(dot_product_results[i]) for i in sorted_indices])
        for i in range(num_top_transitions):
            stage_index = sorted_indices[i]
            stage_name = corresponding_stages[stage_index]
            exponentiated_result = exponentiated_results[stage_index]
            fraction = exponentiated_result / (sum_dot_products + 1)

            if deal_id not in results:
                results[deal_id] = []

            results[deal_id].append({
                "current_stage": current_stage,
                "destination_stage": stage_name,
                "transition_probability": fraction
            })

    return results

transitions = compute_transitions(active_deals_metrics_df, coefficients_all_stages, intercepts_all_stages, num_top_transitions=5)

# Print the results outside the function
for deal_id, transitions_info in transitions.items():
    for transition_info in transitions_info:
        print(f"Deal ID: {deal_id}")
        print(f"Probability of transition from Current Stage: {transition_info['current_stage']} to Destination Stage: {transition_info['destination_stage']}: {transition_info['transition_probability']}")
        print()


Deal ID: 1
Probability of transition from Current Stage: Pitching to Destination Stage: Negotiation: 0.6342806858269073

Deal ID: 1
Probability of transition from Current Stage: Pitching to Destination Stage: Interaction: 0.21385239855770147

Deal ID: 1
Probability of transition from Current Stage: Pitching to Destination Stage: Commitment: 0.07006289061206489

Deal ID: 1
Probability of transition from Current Stage: Pitching to Destination Stage: Closed Lost: 0.03883707780358873

Deal ID: 1
Probability of transition from Current Stage: Pitching to Destination Stage: Closed_Lost: 0.0214834735998687

Deal ID: 6
Probability of transition from Current Stage: Commitment to Destination Stage: Closed Won: 0.9998439226998209

Deal ID: 6
Probability of transition from Current Stage: Commitment to Destination Stage: Interaction: 0.00015607729764306974

Deal ID: 6
Probability of transition from Current Stage: Commitment to Destination Stage: Closed_Lost: 8.453247866739446e-13

Deal ID: 6
Probabi

24/04/07 19:57:19 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1049571 ms exceeds timeout 120000 ms
24/04/07 19:57:19 WARN SparkContext: Killing executors is not supported by current scheduler.
24/04/07 20:00:31 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at 

In [14]:
from pyspark.sql.functions import lag, col, count

stages = ["Interaction", "Pitching", "Negotiation", "Commitment", "Closed Won", "Closed Lost"]
absorbing_stages = {"Closed Won": 1, "Closed Lost": 1} 

def prepare_and_calculate_transitions(df, owner_id=None):
    # Filter by OwnerId if specified
    if owner_id is not None:
        df_filtered = df.filter(df["OwnerId"] == owner_id)
    else:
        df_filtered = df

    # Ensure chronological order within each DealID
    windowSpec = Window.partitionBy("DealID").orderBy("PERIOD")
    df_filtered = df_filtered.withColumn("PrevStage", lag("Opportunity Stage").over(windowSpec))

    # Remove rows where PrevStage is null (the first stage of each DealID)
    transitions_df = df_filtered.filter(col("PrevStage").isNotNull())

    # Calculate counts of transitions from PrevStage to current Opportunity Stage
    transitions_counts = transitions_df.groupBy("PrevStage", "Opportunity Stage").count()

    return transitions_counts

transition_counts=prepare_and_calculate_transitions(df,selected_owner_id)


In [15]:
def calculate_transition_probabilities(transitions_counts):
    # Calculate total transitions out of each stage
    total_transitions_out = transitions_counts.groupBy("PrevStage").agg(F.sum("count").alias("TotalOutgoingTransitions"))

    # Join to get the total outgoing transitions for each PrevStage
    transition_probabilities = transitions_counts.join(
        total_transitions_out,
        "PrevStage"
    ).withColumn(
        "TransitionProbability",
        F.col("count") / F.col("TotalOutgoingTransitions")
    )

    return transition_probabilities

transition_probabilities=calculate_transition_probabilities(transition_counts)


In [16]:
def create_transition_matrix(transition_probabilities, stages, absorbing_stages):
    # Pivot the DataFrame to create a matrix format
    transition_matrix_df = transition_probabilities.groupBy("PrevStage").pivot("Opportunity Stage").agg(F.first("TransitionProbability"))
    
    # Convert to Pandas DataFrame for easier manipulation
    transition_matrix_pd = transition_matrix_df.toPandas()

    # Set index for the DataFrame
    transition_matrix_pd.set_index('PrevStage', inplace=True)

    # Reindex the DataFrame to include all stages, fill missing probabilities with 0
    transition_matrix_pd = transition_matrix_pd.reindex(index=stages, columns=stages, fill_value=0)

    # Fill any remaining NaN values with 0. This is crucial for handling cases where the fill_value=0 in reindex might not apply
    transition_matrix_pd.fillna(0, inplace=True)

    # Adjust for absorbing states: ensure they only transition to themselves
    for stage in stages:
        if stage in absorbing_stages:
            # Absorbing states transition to themselves with probability 1
            transition_matrix_pd.loc[stage, :] = 0
            transition_matrix_pd.loc[stage, stage] = 1
        # The else part is not needed anymore as NaN handling and reindex with fill_value=0 takes care of setting zeros

    return transition_matrix_pd



transition_matrix_pd=create_transition_matrix(transition_probabilities,stages,absorbing_stages)

In [17]:
def print_transition_matrix(transition_matrix_pd):
    print("Transition Probability Matrix:")
    print(transition_matrix_pd)

print(transition_matrix_pd)
transition_matrix_np = transition_matrix_pd.to_numpy()
print(transition_matrix_np)

             Interaction  Pitching  Negotiation  Commitment  Closed Won  \
PrevStage                                                                 
Interaction     0.000000  0.041667     0.027778    0.902778    0.000000   
Pitching        0.236842  0.000000     0.460526    0.144737    0.013158   
Negotiation     0.532468  0.324675     0.000000    0.012987    0.025974   
Commitment      0.187500  0.031250     0.031250    0.000000    0.125000   
Closed Won      0.000000  0.000000     0.000000    0.000000    1.000000   
Closed Lost     0.000000  0.000000     0.000000    0.000000    0.000000   

             Closed Lost  
PrevStage                 
Interaction     0.027778  
Pitching        0.144737  
Negotiation     0.103896  
Commitment      0.625000  
Closed Won      0.000000  
Closed Lost     1.000000  
[[0.         0.04166667 0.02777778 0.90277778 0.         0.02777778]
 [0.23684211 0.         0.46052632 0.14473684 0.01315789 0.14473684]
 [0.53246753 0.32467532 0.         0.01298701

In [18]:
import numpy as np

def calculate_fundamental_matrix(transition_probabilities, absorbing_stages):
    n = len(transition_probabilities)
    non_absorbing_states = [i for i in range(n) if i not in absorbing_stages]
    Q = transition_probabilities[non_absorbing_states][:, non_absorbing_states]
    print("Matrix Q:")
    print(Q)
    I = np.eye(Q.shape[0])
    
    # Display I-Q before taking its inverse
    I_minus_Q = I - Q
    print("\nMatrix I - Q:")
    print(I_minus_Q)
    
    # Take the inverse of I-Q
    N = np.linalg.inv(I_minus_Q)
    
    return N

# Example absorbing stages (replace this with your actual absorbing stages)
absorbing_stages = [4, 5]  # Example: The fifth and sixth stages are absorbing

# Calculate the fundamental matrix using transition_matrix_np
N = calculate_fundamental_matrix(transition_matrix_np, absorbing_stages)

# Print the fundamental matrix
print("\nFundamental Matrix:")
print(N)


Matrix Q:
[[0.         0.04166667 0.02777778 0.90277778]
 [0.23684211 0.         0.46052632 0.14473684]
 [0.53246753 0.32467532 0.         0.01298701]
 [0.1875     0.03125    0.03125    0.        ]]

Matrix I - Q:
[[ 1.         -0.04166667 -0.02777778 -0.90277778]
 [-0.23684211  1.         -0.46052632 -0.14473684]
 [-0.53246753 -0.32467532  1.         -0.01298701]
 [-0.1875     -0.03125    -0.03125     1.        ]]

Fundamental Matrix:
[[1.33822414 0.13962928 0.1399179  1.23014563]
 [0.81291331 1.26932053 0.6360695  0.92585815]
 [0.9804803  0.48751812 1.28213701 0.97236879]
 [0.30696058 0.0810817  0.08617856 1.2899719 ]]


In [19]:
import numpy as np


stages = ["Interaction", "Pitching", "Negotiation", "Commitment", "Closed Won", "Closed Lost"]

# Define absorbing stages
absorbing_stages = [4, 5]  # Closed Won and Closed Lost are absorbing stages

# Function to calculate matrix R
def calculate_matrix_R(transition_probabilities, absorbing_stages):
    n = len(transition_probabilities)
    non_absorbing_states = [i for i in range(n) if i not in absorbing_stages]
    absorbing_states = [i for i in range(n) if i in absorbing_stages]

    # Extract the transition probabilities from non-absorbing states to absorbing states
    R = transition_probabilities[non_absorbing_states][:, absorbing_states]
    return R

R=calculate_matrix_R(transition_matrix_np, absorbing_stages)
# Function to calculate matrix M and describe each entry
def calculate_matrix_M(N, R):
    # Multiply N by R
    M = np.dot(N, R)

    # Describe each entry in matrix M
    description = {}
    for i in range(N.shape[0]):
        for j in range(R.shape[1]):
            description[f"From {stages[i]} to {stages[j + len(stages) - len(absorbing_stages)]}"] = M[i, j]
            if j + len(stages) - len(absorbing_stages) in absorbing_stages:
                description[f"From {stages[i]} to Absorbing State {stages[j + len(stages) - len(absorbing_stages)]}"] = M[i, j]

    return M, description

# Calculate matrix M using provided N and R matrices
matrix_M, description_M = calculate_matrix_M(N, R)

# Print matrix M
print("Matrix M:")
print(matrix_M)

# Print description of each entry in matrix M
print("\nDescription of each entry in matrix M:")
for transition, probability in description_M.items():
    print(f"{transition}: {probability}")




Matrix M:
[[0.15923966 0.84076034]
 [0.14895514 0.85104486]
 [0.16126307 0.83873693]
 [0.16455176 0.83544824]]

Description of each entry in matrix M:
From Interaction to Closed Won: 0.15923966237639206
From Interaction to Absorbing State Closed Won: 0.15923966237639206
From Interaction to Closed Lost: 0.840760337623608
From Interaction to Absorbing State Closed Lost: 0.840760337623608
From Pitching to Closed Won: 0.1489551408846777
From Pitching to Absorbing State Closed Won: 0.1489551408846777
From Pitching to Closed Lost: 0.8510448591153222
From Pitching to Absorbing State Closed Lost: 0.8510448591153222
From Negotiation to Closed Won: 0.16126307058900186
From Negotiation to Absorbing State Closed Won: 0.16126307058900186
From Negotiation to Closed Lost: 0.8387369294109982
From Negotiation to Absorbing State Closed Lost: 0.8387369294109982
From Commitment to Closed Won: 0.164551755804126
From Commitment to Absorbing State Closed Won: 0.164551755804126
From Commitment to Closed Lost:

In [20]:
import numpy as np

def compute_transition_matrix_power(transition_matrix, power=None):
    if power is None:
        # If power is not specified, use the power corresponding to the steady state
        # The power is the minimum number of steps required to reach the steady state
        eigenvalues, eigenvectors = np.linalg.eig(transition_matrix)
        steady_state_index = np.argmin(np.abs(eigenvalues - 1))  # Index of the eigenvalue closest to 1
        power = int(np.ceil(1 / np.abs(eigenvalues[steady_state_index])))  # Power corresponding to the steady state

    # Compute the transition matrix raised to the specified power
    transition_matrix_power = np.linalg.matrix_power(transition_matrix, power)

    return transition_matrix_power

# Usage:
# 1. Compute the transition matrix (transition_matrix_np)

# 2. Compute the transition matrix to the desired power (default is the power corresponding to the steady state)
transition_matrix_power = compute_transition_matrix_power(transition_matrix_np)

# 3. Use the transition matrix to the desired power for further computations
# (No changes needed as it's already implemented)



print(transition_matrix_power)
# 3. Use the cumulative transition matrix for further computations
# (No changes needed as it's already implemented)


[[0.         0.04166667 0.02777778 0.90277778 0.         0.02777778]
 [0.23684211 0.         0.46052632 0.14473684 0.01315789 0.14473684]
 [0.53246753 0.32467532 0.         0.01298701 0.02597403 0.1038961 ]
 [0.1875     0.03125    0.03125    0.         0.125      0.625     ]
 [0.         0.         0.         0.         1.         0.        ]
 [0.         0.         0.         0.         0.         1.        ]]


In [21]:
# Iterate through the computed transitions
for deal_id, transitions_info in transitions.items():
    for transition_info in transitions_info:
        # Extract information
        fraction = transition_info['transition_probability']
        destination_stage = transition_info['destination_stage']
        
        # Adjust the destination stage name if needed
        if destination_stage == "Closed_Won":
            destination_stage = "Closed Won"
        elif destination_stage == "Closed_Lost":
            destination_stage = "Closed Lost"
        
        # Check if the destination stage exists in the index of the transition matrix
        if destination_stage in transition_matrix_pd.index:
            # Find the row index corresponding to the destination stage
            destination_stage_index = np.where(transition_matrix_pd.index == destination_stage)[0]
            if len(destination_stage_index) > 0:
                destination_stage_index = destination_stage_index[0]
                
                # Check if the destination stage is "Closed Lost" or "Closed Won"
                if destination_stage == "Closed Lost":
                    # Exceptional case: Multiply by 1 if destination stage is "Closed Lost"
                    score1 = fraction * 1
                    score2 = fraction * 1
                    print(f"Calculating CRM score for {destination_stage}")
                elif destination_stage == "Closed Won":
                    # Exceptional case: Multiply by 1 if destination stage is "Closed Won"
                    score1 = fraction * 1
                    score2 = fraction * 1
                    print(f"Calculating CRM score for {destination_stage}")
                else:
                    # Find the entries in the first and second columns of the transition matrix
                    # Note: Assuming destination stage columns are the first and second columns in matrix M
                    entry1 = matrix_M[destination_stage_index, 0]
                    entry2 = matrix_M[destination_stage_index, 1]
                    # Multiply the fraction by the entries
                    score1 = fraction * entry1
                    score2 = fraction * entry2
                    # Print the fraction and entries before they are multiplied together
                    print(f"Fraction: {fraction}")
                    print(f"Entry 1 before multiplication for {destination_stage}: {entry1}")
                    print(f"Entry 2 before multiplication for {destination_stage}: {entry2}")
                
                # Print the information for the current transition
                print(f"Deal ID: {deal_id}")
                print(f"Final CRM score 1 for {destination_stage}: {score1}")
                print(f"Final CRM score 2 for {destination_stage}: {score2}")
                print(f"Current Stage: {transition_info['current_stage']}")
                print(f"Destination Stage: {destination_stage}")
                print()
            else:
                # Print a message indicating that the destination stage is not found in the transition matrix index
                print(f"Destination stage '{destination_stage}' not found in the transition matrix index.")
        else:
            # Print a message indicating that the destination stage is not found in the transition matrix index
            print(f"Destination stage '{destination_stage}' not found in the transition matrix index.")


Fraction: 0.6342806858104473
Entry 1 before multiplication for Negotiation: 0.16126307058900186
Entry 2 before multiplication for Negotiation: 0.8387369294109982
Deal ID: 1
Final CRM score 1 for Negotiation: 0.10228605100909066
Final CRM score 2 for Negotiation: 0.5319946348013567
Current Stage: Pitching
Destination Stage: Negotiation

Fraction: 0.2138523985524389
Entry 1 before multiplication for Interaction: 0.15923966237639206
Entry 2 before multiplication for Interaction: 0.840760337623608
Deal ID: 1
Final CRM score 1 for Interaction: 0.034053783743872
Final CRM score 2 for Interaction: 0.1797986148085669
Current Stage: Pitching
Destination Stage: Interaction

Fraction: 0.07006289061082381
Entry 1 before multiplication for Commitment: 0.164551755804126
Entry 2 before multiplication for Commitment: 0.835448244195874
Deal ID: 1
Final CRM score 1 for Commitment: 0.011528971666723472
Final CRM score 2 for Commitment: 0.058533918944100344
Current Stage: Pitching
Destination Stage: Commi

In [22]:
# Initialize dictionaries to store the sum of Final CRM score 1s and 2s for each destination stage within each deal ID
final_crm_score_1_sums = {}
final_crm_score_2_sums = {}

# Iterate through the computed transitions
for deal_id, transitions_info in transitions.items():
    # Initialize total sums for this deal ID
    total_score_1 = 0
    total_score_2 = 0
    
    for transition_info in transitions_info:
        # Extract information
        fraction = transition_info['transition_probability']
        destination_stage = transition_info['destination_stage']
        
        # Adjust the destination stage name if needed
        if destination_stage == "Closed_Won":
            destination_stage = "Closed Won"
        elif destination_stage == "Closed_Lost":
            destination_stage = "Closed Lost"
        
        # Check if the destination stage exists in the index of the transition matrix
        if destination_stage in transition_matrix_pd.index:
            # Find the row index corresponding to the destination stage
            destination_stage_index = np.where(transition_matrix_pd.index == destination_stage)[0]
            if len(destination_stage_index) > 0:
                destination_stage_index = destination_stage_index[0]
                
                # Check if the destination stage is "Closed Lost" or "Closed Won"
                if destination_stage == "Closed Lost" or destination_stage == "Closed Won":
                    # Exceptional case: Multiply by 1 if destination stage is "Closed Lost" or "Closed Won"
                    score1 = fraction * 1
                    score2 = fraction * 1
                else:
                    # Find the entries in the first and second columns of the transition matrix
                    entry1 = matrix_M[destination_stage_index, 0]
                    entry2 = matrix_M[destination_stage_index, 1]
                    # Multiply the fraction by the entries
                    score1 = fraction * entry1
                    score2 = fraction * entry2
                
                # Add scores to the total sums for this deal ID
                total_score_1 += score1
                total_score_2 += score2
                
            else:
                # Print a message indicating that the destination stage is not found in the transition matrix index
                print(f"Destination stage '{destination_stage}' not found in the transition matrix index.")
        else:
            # Print a message indicating that the destination stage is not found in the transition matrix index
            print(f"Destination stage '{destination_stage}' not found in the transition matrix index.")
    
    # Store the total sums for this deal ID
    final_crm_score_1_sums[deal_id] = total_score_1
    final_crm_score_2_sums[deal_id] = total_score_2

# Print the total sums of Final CRM score 1 and 2 for each deal ID
for deal_id in final_crm_score_1_sums.keys():
    print(f"Deal ID: {deal_id}")
    print(f"Sum of Final CRM score 1: {final_crm_score_1_sums[deal_id]}")
    print(f"Sum of Final CRM score 2: {final_crm_score_2_sums[deal_id]}")
    print()



# Convert final_crm_score_1_sums to a NumPy array
final_crm_score_1_array = np.array(list(final_crm_score_1_sums.values()))

print(final_crm_score_1_array)

# Print the total sums of Final CRM score 1 and 2 for each deal ID

# Now you can use final_crm_score_1_array for dot product with latest_avg_contract_values_array
dot_product_result = np.dot(final_crm_score_1_array, latest_avg_contract_values_array)
print("Dot Product Result:", dot_product_result)


# Find the indices of the top 5 deals with the highest average contract value
top_5_indices = np.argsort(-latest_avg_contract_values_array)[:5]

# Get the deal IDs of the top 5 deals
top_5_deal_ids = [row["DealID"] for row in active_deals_metrics_df.collect()]

# Display the CRM score 1 and average contract value for the top 5 deals
print("Top 5 Deals with Highest Average Contract Value:")
for index in top_5_indices:
    deal_id = top_5_deal_ids[index]
    print(f"Deal ID: {deal_id}")
    print(f"CRM Score 1: {final_crm_score_1_sums.get(deal_id, 'N/A')}")
    print(f"Average Contract Value: {latest_avg_contract_values_array[index]}")
    print()



Deal ID: 1
Sum of Final CRM score 1: 0.20818935783408218
Sum of Final CRM score 2: 0.8306477199684199

Deal ID: 6
Sum of Final CRM score 1: 0.9998687763976452
Sum of Final CRM score 2: 0.9999751463029644

Deal ID: 8
Sum of Final CRM score 1: 0.15366545926304834
Sum of Final CRM score 2: 0.8465439244655063

Deal ID: 11
Sum of Final CRM score 1: 0.15923966237639206
Sum of Final CRM score 2: 0.840760337623608

Deal ID: 14
Sum of Final CRM score 1: 0.164551755804126
Sum of Final CRM score 2: 0.835448244195874

Deal ID: 24
Sum of Final CRM score 1: 0.9999999999988441
Sum of Final CRM score 2: 0.999999999999648

Deal ID: 25
Sum of Final CRM score 1: 1.0
Sum of Final CRM score 2: 1.0

Deal ID: 30
Sum of Final CRM score 1: 0.15923966237639206
Sum of Final CRM score 2: 0.840760337623608

Deal ID: 32
Sum of Final CRM score 1: 0.999998593652493
Sum of Final CRM score 2: 0.9999997344918108

Deal ID: 37
Sum of Final CRM score 1: 0.15923966237639206
Sum of Final CRM score 2: 0.840760337623608

Deal 

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
import numpy as np

def calculate_all_point_biserial_correlations(df, metrics, owner_id=None, binary_col='Closed won'):
    """
    Calculates the point-biserial correlation coefficient for a list of metrics against a binary variable.
    Can filter by OwnerId or use the entire dataset. The correlation coefficients are converted to percentages.

    Parameters:
    - df: PySpark DataFrame containing the dataset.
    - metrics: List of metric names (strings) to calculate correlations for.
    - owner_id: (Optional) The OwnerId to filter the dataset by. If None, uses the entire dataset.
    - binary_col: Name of the binary column to calculate correlation against.

    Returns:
    - A dictionary with each metric as keys and their point-biserial correlation with 'Closed won' as percentage values.
    """
    if owner_id is not None:
        df_filtered = df.filter(F.col('OwnerId') == owner_id)  # Filter by OwnerId if specified
    else:
        df_filtered = df  # Use the entire dataset if no OwnerId is specified

    correlations = {}
    for metric in metrics:
        mean_1 = df_filtered.filter(F.col(binary_col) == 1).agg({metric: 'mean'}).collect()[0][0]
        mean_0 = df_filtered.filter(F.col(binary_col) == 0).agg({metric: 'mean'}).collect()[0][0]
        std_dev = df_filtered.agg({metric: 'stddev'}).collect()[0][0]
        n_1 = df_filtered.filter(F.col(binary_col) == 1).count()
        n_0 = df_filtered.filter(F.col(binary_col) == 0).count()
        n = df_filtered.count()
        
        if std_dev is not None and std_dev > 0 and n_1 > 0 and n_0 > 0:  # Ensure valid calculation
            r_pb = ((mean_1 - mean_0) / std_dev) * np.sqrt((n_1 * n_0) / float(n)**2)
            correlations[metric] = r_pb * 100  # Convert correlation to percentage
        else:
            correlations[metric] = None  # Handle cases where calculation cannot be performed
    
    return correlations

# Define your metrics list and use the function as needed
metrics = [
    "Inbound Calls", "Inbound Email", "Outbound Email", "Meetings Scheduled",
    "Lead Response Time", "Average Contract Value", "Sales Cycle Length",
    "Linkedin Messaging", "Daily opportunity movements", "Outbound leads created",
    "Contacts per/in company", "Sales by territory", "Sales by industry", "Sales by channel",
    "Number of Demos"
]

def rank_correlations(correlations):
    """
    Sorts the correlations dictionary from highest to lowest correlation values.

    Parameters:
    - correlations: Dictionary with metrics as keys and correlation values as values.

    Returns:
    - A sorted list of tuples (metric, correlation) from highest to lowest correlation.
    """
    # Filter out None values, sort by correlation value in descending order
    sorted_correlations = sorted([(metric, corr) for metric, corr in correlations.items() if corr is not None], key=lambda x: x[1], reverse=True)
    return sorted_correlations

# Example usage
correlations_entire_dataset = calculate_all_point_biserial_correlations(df, metrics)
ranked_correlations_dataset = rank_correlations(correlations_entire_dataset)
print(ranked_correlations_dataset)


In [ ]:
# Example usage
  # To analyze data for a specific OwnerId
correlations_specific_owner = calculate_all_point_biserial_correlations(df, metrics, selected_owner_id)
ranked_correlations_specific_owner = rank_correlations(correlations_specific_owner)

print ('Ranked correlations for Owner ID',selected_owner_id,ranked_correlations_specific_owner)

In [ ]:
from pyspark.sql import SparkSession, functions as F, Window
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# calculation of the signals function by performing a binary logistic regression. Can be done both dataset/company wide and OwnerID wide

def analyze_data_and_train_model(raw_df, owner_id=None):
    # Filter the DataFrame by OwnerId if specified
    if owner_id is not None:
        df_filtered = raw_df.filter(raw_df['OwnerId'] == owner_id)
    else:
        df_filtered = raw_df

    # Define the window specification for cumulative sum
    windowSpec = Window.partitionBy("DealID").orderBy("PERIOD").rowsBetween(Window.unboundedPreceding, Window.currentRow)

    # List of metrics to include in cumulative sum
    metrics = [
        "Inbound Calls", "Inbound Email", "Outbound Email", "Meetings Scheduled",
        "Lead Response Time", "Average Contract Value", "Sales Cycle Length",
        "Linkedin Messaging", "Daily opportunity movements", "Outbound leads created",
        "Contacts per/in company", "Sales by territory", "Sales by industry", "Sales by channel",
        "Number of Demos", "Closed won"
    ]

    # Calculate cumulative sum for each metric within each DealID
    for metric in metrics:
        df_filtered = df_filtered.withColumn(f"{metric}_cumulative", F.sum(F.col(metric)).over(windowSpec))

    # Select only the last row for each DealID after cumulative calculation
    windowSpecLastRow = Window.partitionBy("DealID").orderBy(F.desc("PERIOD"))
    df_cumulative = df_filtered.withColumn("row", F.row_number().over(windowSpecLastRow)) \
                               .filter(F.col("row") == 1) \
                               .drop("row")

    # Prepare data for logistic regression
    featureCols = [col for col in df_cumulative.columns if col.endswith("_cumulative") and col != "Closed won_cumulative"]
    assembler = VectorAssembler(inputCols=featureCols, outputCol="features")
    df_assembled = assembler.transform(df_cumulative)
    df_model = df_assembled.select(F.col("features"), F.col("Closed won_cumulative").alias("label"))

    # Train the logistic regression model
    lr = LogisticRegression(featuresCol="features", labelCol="label")
    model = lr.fit(df_model)

    # Make predictions on the dataset
    predictions = model.transform(df_model)

    # Evaluate the model
    evaluator = BinaryClassificationEvaluator(labelCol="label")
    accuracy = evaluator.evaluate(predictions)
    print(f"Model Accuracy: {accuracy}")

    # Calculate and display odds ratios
    coefficients = model.coefficients.toArray()
    odds_ratios = (np.exp(coefficients) * 100)-100
    metric_odds = zip(featureCols, odds_ratios)
    
    # Sort by odds ratio in descending order
    sorted_metric_odds = sorted(metric_odds, key=lambda x: x[1], reverse=True)
    
    print("Odds Ratios for Each Metric (as percentages), Ranked from Highest to Lowest:")
    for name, odds_ratio in sorted_metric_odds:
        print(f"{name}: {odds_ratio:.2f}%")

    return model, accuracy


specific_owner_id = None
model, accuracy = analyze_data_and_train_model(raw_df)
# Example usage
  # Use this to analyze data for a specific OwnerId
# specific_owner_id = None  # Uncomment this and comment out the above line to analyze the entire dataset








In [ ]:

model_for_owner, accuracy_for_owner = analyze_data_and_train_model(raw_df,selected_owner_id)

In [ ]:

from pyspark.sql.window import Window
import pyspark.sql.functions as F
from itertools import chain
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis

metrics = [
    "Inbound Calls", "Inbound Email", "Outbound Email", "Meetings Scheduled",
    "Lead Response Time", "Average Contract Value", "Sales Cycle Length",
    "Linkedin Messaging", "Daily opportunity movements", "Outbound leads created",
    "Contacts per/in company", "Sales by territory", "Sales by industry", "Sales by channel",
    "Number of Demos"
]


def calculate_metrics_stats(df, metrics, OwnerId=None):
    """
    Calculates statistical metrics (mean, skewness, kurtosis, max, min, PMF) for each metric in 'metrics'.
    Optionally filters the dataset for a specific OwnerId before calculation.
    
    Parameters:
    - df: Spark DataFrame containing the dataset for analysis.
    - metrics: List of metrics (column names in 'df') for which to calculate the statistics.
    - OwnerId: Optional. If provided, the dataset will be filtered to include only data for this OwnerId.
    
    Returns:
    A dictionary of statistics for each metric.
    """
    if OwnerId is not None:
        df = df.filter(df["OwnerId"] == OwnerId)
    
    deal_window = Window.partitionBy("OwnerId", "DealID").orderBy("Period")
    
    for metric in metrics:
        df = df.withColumn(f"Cumulative_{metric}", F.sum(metric).over(deal_window))
    
    results = {}
    
    for metric in metrics:
        cum_metric = f"Cumulative_{metric}"
        pd_df = df.select("OwnerId", cum_metric).toPandas()
        
        results[metric] = {
            "Max": pd_df[cum_metric].max(),
            "Min": pd_df[cum_metric].min(),
            "Average": pd_df[cum_metric].mean(),
            "Skewness": skew(pd_df[cum_metric], bias=False),
            "Kurtosis": kurtosis(pd_df[cum_metric], bias=False),
            "PMF": (pd_df[cum_metric].value_counts().sort_index() / len(pd_df)).to_dict()
        }
    
    return results


In [ ]:
def print_metrics_stats(results):
    """
    Prints the calculated statistics for each metric in a readable format.
    
    Parameters:
    - results: A dictionary of statistics for each metric, as returned by calculate_metrics_stats().
    """
    for metric, stats in results.items():
        print(f"---{metric}---")
        print(f"Maximum: {stats['Max']}")
        print(f"Minimum: {stats['Min']}")
        print(f"Average: {stats['Average']}")
        print(f"Skewness: {stats['Skewness']}")
        print(f"Kurtosis: {stats['Kurtosis']}")
        print("PMF:")
        for value, probability in stats['PMF'].items():
            print(f"  Value: {value}, Probability: {probability}")
        print("\n")

# Example Usage:
results = calculate_metrics_stats(df, metrics)
print_metrics_stats(results)


In [ ]:
results_for_selected_owner_id=calculate_metrics_stats(df,metrics,selected_owner_id)
print_metrics_stats(results_for_selected_owner_id)

In [ ]:
def calculate_elapsed_days_stats(df, OwnerId=None):
    """
    Calculates statistics for the elapsed days across DealIDs within the dataset.
    Optionally filters the dataset for a specific OwnerId before calculation.
    
    Parameters:
    - df: Spark DataFrame containing the dataset for analysis.
    - OwnerId: Optional. If provided, the dataset will be filtered to include only data for this OwnerId.
    
    Returns:
    A dictionary of statistics for the elapsed days.
    """
    # If OwnerId is specified, filter the DataFrame to include only rows for that OwnerId
    if OwnerId is not None:
        df = df.filter(df["OwnerId"] == OwnerId)
    
    deal_dates = df.groupBy("DealID").agg(
        min("PERIOD").alias("Start_Date"),
        max("PERIOD").alias("End_Date")
    )
    
    # Calculate the time elapsed in days for each DealID
    deal_dates = deal_dates.withColumn("Days_Elapsed", datediff("End_Date", "Start_Date"))
    
    # Convert to Pandas DataFrame for statistical calculations
    deal_dates_pd = deal_dates.select("Days_Elapsed").toPandas()
    
    stats = {
        "Max": deal_dates_pd["Days_Elapsed"].max(),
        "Min": deal_dates_pd["Days_Elapsed"].min(),
        "Average": deal_dates_pd["Days_Elapsed"].mean(),
        "Skewness": skew(deal_dates_pd["Days_Elapsed"], bias=False),
        "Kurtosis": kurtosis(deal_dates_pd["Days_Elapsed"], bias=False),
        "PMF": (deal_dates_pd["Days_Elapsed"].value_counts().sort_index() / len(deal_dates_pd)).to_dict()
    }
    
    return stats


def print_elapsed_days_stats(stats):
    """
    Prints the calculated statistics for elapsed days in a readable format.
    
    Parameters:
    - stats: A dictionary of statistics for elapsed days, as returned by calculate_elapsed_days_stats().
    """
    print(f"Maximum Elapsed Days: {stats['Max']}")
    print(f"Minimum Elapsed Days: {stats['Min']}")
    print(f"Average Elapsed Days: {stats['Average']}")
    print(f"Skewness of Elapsed Days: {stats['Skewness']}")
    print(f"Kurtosis of Elapsed Days: {stats['Kurtosis']}")
    print("PMF of Elapsed Days:")
    for value, prob in stats['PMF'].items():
        print(f"  Days: {value}, Probability: {prob}")


stats_elapsed_days = calculate_elapsed_days_stats(df)
print_elapsed_days_stats(stats_elapsed_days)


In [ ]:
# Assuming OwnerId is '123'

stats_elapsed_days_selected_owner_id = calculate_elapsed_days_stats(df, selected_owner_id)
print_elapsed_days_stats(stats_elapsed_days_selected_owner_id)  # Use the same print function as defined previously


In [ ]:

#function which calculates mean,skewness,kurtosis, and probability mass function
# for the number of elapsed time steps for a deal both OwnerID specific and company wide specific



def calculate_deal_transitions_statistics_debug(df, owner_id=None):
    if owner_id is not None:
        df_filtered = df.filter(df["OwnerId"] == owner_id)
    else:
        df_filtered = df
    transitions_count = df.groupBy("DealID").count().withColumnRenamed("count", "Transitions")
    transitions_pd = transitions_count.select("Transitions").toPandas()
    
    max_transitions = transitions_pd["Transitions"].max()
    min_transitions = transitions_pd["Transitions"].min()
    avg_transitions = transitions_pd["Transitions"].mean()
    
    # Check variance before calculating skewness and kurtosis
    if transitions_pd["Transitions"].var() == 0:  # No variance
        skew_transitions = None
        kurt_transitions = None
    else:
        skew_transitions = skew(transitions_pd["Transitions"], bias=False)
        kurt_transitions = kurtosis(transitions_pd["Transitions"], bias=False)
    
    pmf_transitions = transitions_pd["Transitions"].value_counts().sort_index() / len(transitions_pd)
    
    statistics = {
        "Maximum Transitions": max_transitions,
        "Minimum Transitions": min_transitions,
        "Average Transitions": avg_transitions,
        "Skewness of Transitions": skew_transitions,
        "Kurtosis of Transitions": kurt_transitions,
        "PMF of Transitions": pmf_transitions.to_dict()
    }
    
    return statistics

# Example usage remains the same



# Call the function and store the results



def print_deal_transitions_statistics(stats):
    """
    Prints the statistics for the number of transitions per deal in a readable format.

    Parameters:
    - stats: A dictionary containing statistics for deal transitions,
             as returned by calculate_deal_transitions_statistics_debug().
    """
    print(f"Maximum Transitions: {stats['Maximum Transitions']}")
    print(f"Minimum Transitions: {stats['Minimum Transitions']}")
    print(f"Average Transitions: {stats['Average Transitions']}")
    
    skew_transitions = stats['Skewness of Transitions']
    if skew_transitions is not None:
        print(f"Skewness of Transitions: {skew_transitions}")
    else:
        print("Skewness of Transitions: Not calculated due to zero variance.")
    
    kurt_transitions = stats['Kurtosis of Transitions']
    if kurt_transitions is not None:
        print(f"Kurtosis of Transitions: {kurt_transitions}")
    else:
        print("Kurtosis of Transitions: Not calculated due to zero variance.")
    
    print("PMF of Transitions:")
    for value, probability in stats['PMF of Transitions'].items():
        print(f"  Transitions: {value}, Probability: {probability}")

stats_elapsed_time_steps = calculate_deal_transitions_statistics_debug(df)
print_deal_transitions_statistics(stats_elapsed_time_steps)


In [ ]:
stats_elapsed_transitions_selected_owner_id = calculate_deal_transitions_statistics_debug(df,selected_owner_id)
print_deal_transitions_statistics(stats_elapsed_transitions_selected_owner_id)

In [ ]:
from pyspark.sql.functions import col, sum, avg, dense_rank, first, row_number
from pyspark.sql.window import Window

def calculate_elapsed_transitions_with_latest_stage(df, owner_id=None):
    if owner_id is not None:
        df = df.filter(col("OwnerId") == owner_id)
    
    windowSpecChrono = Window.partitionBy("DealID").orderBy("PERIOD")
    windowSpecChronoDesc = Window.partitionBy("DealID").orderBy(col("PERIOD").desc())
    
    df = df.withColumn("CumulativeClosedWon", sum(col("Closed won")).over(windowSpecChrono))\
           .withColumn("CumulativeClosedLost", sum(col("Closed lost")).over(windowSpecChrono))
    
    df_filtered = df.filter((col("CumulativeClosedWon") == 0) & (col("CumulativeClosedLost") == 0))

    # Aggregate to get the average "Average Contract Value" and count the number of transitions
    df_agg = df_filtered.groupBy("DealID")\
                        .agg(
                            avg(col("Average Contract Value")).alias("AvgContractValue"),
                            count("*").alias("NumTransitions")
                        )

    windowSpecRanking = Window.orderBy(col("AvgContractValue").desc())
    df_agg_ranked = df_agg.withColumn("Rank", dense_rank().over(windowSpecRanking)).filter(col("Rank") <= 5)

    # Use row_number to get the most recent record for each DealID in the filtered set
    df_latest_stage = df_filtered.withColumn("rn", row_number().over(windowSpecChronoDesc))\
                                 .filter(col("rn") == 1).select("DealID", "Opportunity Stage")

    # Join to combine the latest stage with the aggregate results
    result = df_agg_ranked.join(df_latest_stage, "DealID").select("DealID", "NumTransitions", "Opportunity Stage").collect()

    return result

# Assuming df is your DataFrame
elapsed_transitions_with_latest_stage = calculate_elapsed_transitions_with_latest_stage(df, owner_id=None)
for deal in elapsed_transitions_with_latest_stage:
    print(f"DealID: {deal['DealID']}, Elapsed Transitions: {deal['NumTransitions']}, Latest Stage: {deal['Opportunity Stage']}")




In [ ]:
owner_id = "1"

elapsed_transitions_with_latest_stage_for_owner = calculate_elapsed_transitions_with_latest_stage(df, owner_id)
for deal in elapsed_transitions_with_latest_stage:
    print("For Owner ID",owner_id,f"DealID: {deal['DealID']}, Elapsed Transitions: {deal['NumTransitions']}, Latest Stage: {deal['Opportunity Stage']}")

In [ ]:
import numpy as np
from scipy.stats import poisson

# Assuming you've already defined or loaded owner-specific data and transition matrix
stats = calculate_deal_transitions_statistics_debug(df)
# Access individual statistics from the results
maximum_transitions = stats["Maximum Transitions"]
minimum_transitions = stats["Minimum Transitions"]
average_transitions = stats["Average Transitions"]
kurtosis_transitions = stats["Kurtosis of Transitions"]
# PMF is a dictionary itself, you can access it directly or iterate over its items
pmf_transitions = stats["PMF of Transitions"]

def calculate_adjusted_matrix_sum(matrix, stats, elapsed_transitions):
    pmf_transitions = stats["PMF of Transitions"]
    max_transitions = int(stats["Maximum Transitions"])
    probabilities = np.zeros(max_transitions + 1)
    for transition, probability in pmf_transitions.items():
        probabilities[transition] = probability

    print("Original Probabilities:", probabilities)
    
    adjusted_probabilities = probabilities[elapsed_transitions:]
    print("Adjusted Probabilities Before Normalization:", adjusted_probabilities)
    
    if adjusted_probabilities.sum() > 0:
        adjusted_probabilities /= adjusted_probabilities.sum()
    else:
        print("No probabilities to adjust; returning zero matrix.")
        return np.zeros_like(matrix)
    
    print("Normalized Adjusted Probabilities:", adjusted_probabilities)

    weighted_sum_matrix = np.zeros_like(matrix, dtype=float)
    for i, prob in enumerate(adjusted_probabilities, start=1):
        matrix_nth_power = np.linalg.matrix_power(matrix, i)
        print(f"Matrix to the {i}-th power * Probability {prob}:")
        print(matrix_nth_power * prob)
        weighted_sum_matrix += matrix_nth_power * prob
    
    return weighted_sum_matrix

elapsed_transitions_with_latest_stage = calculate_elapsed_transitions_with_latest_stage(df, owner_id=None)




    
    # Further processing using result_matrix...

# Define a mapping from stage names to indices
stage_to_index_mapping = {
    "Interaction": 0,
    "Pitching": 1,
    "Negotiation": 2,
    "Commitment": 3,
    "Closed Won": 4,
    "Closed Lost": 5
}

import numpy as np
from scipy.stats import poisson

# Assuming df is your PySpark DataFrame and it includes a column for "Average Contract Value"

def get_avg_contract_value(deal_id, df):
    # Hypothetical function to retrieve "Average Contract Value" for a given Deal ID
    # Implement according to how your data is structured
    avg_value = df.filter(df['DealID'] == deal_id).select('Average Contract Value').collect()[0][0]
    return avg_value

def get_owner_id(deal_id, df):
    # Hypothetical function to retrieve "OwnerId" for a given Deal ID
    # Implement according to how your data is structured
    owner_id = df.filter(df['DealID'] == deal_id).select('OwnerId').collect()[0][0]
    return owner_id


# Assuming you have a function to generate stats and calculate elapsed transitions
# stats = calculate_deal_transitions_statistics_debug(df)


# Define a mapping from stage names to indices


# Loop through each deal to calculate and print the desired information
for deal in elapsed_transitions_with_latest_stage:
    deal_id = deal['DealID']
    num_transitions = deal['NumTransitions']
    latest_stage_name = deal['Opportunity Stage']
    
    # Fetch the "OwnerId" for the current Deal ID
    owner_id = get_owner_id(deal_id, df)
    
    # Fetch owner-specific statistics for the current Deal ID
    owner_stats = calculate_deal_transitions_statistics_debug(df, owner_id)
    
    # Fetch the owner-specific transition probability matrix for the current Deal ID
    transition_matrix_np = calculate_transition_probabilities(df, specific_owner_id=owner_id)
    
    # Fetch the "Average Contract Value" for the current Deal ID
    avg_contract_value = get_avg_contract_value(deal_id, df)
    
    latest_stage_index = stage_to_index_mapping.get(latest_stage_name, -1)
    
    if latest_stage_index != -1:
        result_matrix = calculate_adjusted_matrix_sum(transition_matrix_np, owner_stats, num_transitions)
        
        if 0 <= latest_stage_index < result_matrix.shape[0]:
            probability_to_closed_won = result_matrix[latest_stage_index, stage_to_index_mapping["Closed Won"]]
            print(f"Owner ID: {owner_id}, DealID: {deal_id}, Stage: {latest_stage_name}, Probability to Closed Won: {probability_to_closed_won:.2f}, Avg Contract Value: {avg_contract_value}")
        else:
            print(f"Owner ID: {owner_id}, DealID: {deal_id}, latest stage index out of bounds.")
    else:
        print(f"Owner ID: {owner_id}, DealID: {deal_id}, invalid latest stage name.")





In [ ]:
import numpy as np
from scipy.stats import poisson
owner_id = '2'  # Specify the owner ID you're interested in

# Filter the DataFrame for this specific owner before performing any calculations
df_owner_specific = df.filter(df['OwnerId'] == owner_id)

# Calculate statistics and transitions for this specific owner
stats_for_owner = calculate_deal_transitions_statistics_debug(df_owner_specific)
elapsed_transitions_with_latest_stage_for_owner = calculate_elapsed_transitions_with_latest_stage(df_owner_specific, owner_id)

# Define a mapping from stage names to indices
stage_to_index_mapping = {
    "Interaction": 0,
    "Pitching": 1,
    "Negotiation": 2,
    "Commitment": 3,
    "Closed Won": 4,
    "Closed Lost": 5
}
def calculate_adjusted_matrix_sum_owner(matrix_owner, stats_owner, elapsed_transitions):
    # Check if stats_owner or matrix_owner is empty or not initialized properly
    if 'PMF of Transitions' not in stats_owner or not len(matrix_owner):
        print("No data available for specified owner or matrix is empty.")
        return np.zeros((6, 6))  # Adjust the size based on your default or expected matrix size
    
    pmf_transitions_owner = stats_owner.get("PMF of Transitions", {})
    max_transitions_owner = int(stats_owner.get("Maximum Transitions", 0))
    probabilities_owner = np.zeros(max_transitions_owner + 1)
    
    for transition, probability in pmf_transitions_owner.items():
        probabilities_owner[transition] = probability

    # If the entire probabilities_owner array is zeros, then it's likely no data was found
    if not probabilities_owner.sum() > 0:
        print("No transition data available for specified owner.")
        return np.zeros_like(matrix_owner)

    adjusted_probabilities_owner = probabilities_owner[elapsed_transitions:]
    print("Adjusted Probabilities Before Normalization:", adjusted_probabilities_owner)
    if adjusted_probabilities_owner.sum() > 0:
        adjusted_probabilities_owner /= adjusted_probabilities_owner.sum()
    else:
        print("No probabilities to adjust; returning zero matrix.")
        return np.zeros_like(matrix_owner)
    
    print("Normalized Adjusted Probabilities:", adjusted_probabilities_owner)
    weighted_sum_matrix_owner = np.zeros_like(matrix_owner, dtype=float)
    for i, prob in enumerate(adjusted_probabilities_owner, start=1):
        matrix_nth_power_owner = np.linalg.matrix_power(matrix_owner, i)
        print(f"Matrix to the {i}-th power * Probability {prob}:")
        print(matrix_nth_power_owner * prob)
        weighted_sum_matrix_owner += matrix_nth_power_owner * prob
    
    return weighted_sum_matrix_owner






# Modify your existing for loop to include the "Average Contract Value"
# Modify your existing for loop to include the "Average Contract Value"
for deal in elapsed_transitions_with_latest_stage_for_owner:
    deal_id = deal['DealID']
    num_transitions = deal['NumTransitions']  # Extract the NumTransitions value
    latest_stage_name = deal['Opportunity Stage']
    # Ensure to use the correct key from your data to fetch the average contract value
    avg_contract_value = get_avg_contract_value(deal_id, df) # Make sure 'Average Contract Value' matches the field name in your data

    latest_stage_index = stage_to_index_mapping.get(latest_stage_name, -1)  # Convert stage name to index
    
    if latest_stage_index != -1:
        # Pass num_transitions instead of the whole deal dictionary
        result_matrix_owner = calculate_adjusted_matrix_sum_owner(transition_matrix_np, stats_for_owner, num_transitions)
        
        if 0 <= latest_stage_index < result_matrix_owner.shape[0]:
            probability_to_closed_won = result_matrix_owner[latest_stage_index, 4]  # "Closed Won" at index 4
            # Include "Average Contract Value" in your print statement
            print(f"For Owner ID {owner_id}, DealID: {deal_id}, Stage: {latest_stage_name}, Probability to Closed Won: {probability_to_closed_won:.2%}, Avg Contract Value: {avg_contract_value}")
        else:
            print(f"For Owner ID {owner_id}, DealID: {deal_id}, latest stage index out of bounds.")
    else:
        print(f"For Owner ID {owner_id}, DealID: {deal_id}, invalid latest stage name.")

